# Yearly run times
Version of the yearly ablation tests that gets the run-times for the final model and architecture ablation tests.
#### Notes
- Only one fold model is created for each run
- Only 20 runs per test - i.e. timings are for one ensemble of twenty runs.
- No parallel running of tests
- The first test (test0) is the proposed model, so other tests are offset by 1 compared to the full ablation test (e.g. the dropout test is test4 here and test3 in the ablation test)
- By default, training times are output to the `train_stats.csv` file, so these could also be obtained from the ablation tests model directories. This notebook allows testing under more controlled conditions - e.g. if the full runs are run in a shared environment they may not be accurate/consistent due to the server workload. So this is a cut-down version that can run in a small dedicated environment.

In [1]:
import os
import json
import numpy as np
import pandas as pd

import initialise
from model_utils import reshape_data
from modelling_functions import create_models, run_experiment
from model_parameters import ModelParams

## Directories and Input files
Change these settings as required
- `input_dir`: Directory containing the data extracted from GEE and Globe-LFMC, the outputs from running the `Extract DEM Data.ipynb` and `Extract MODIS Data.ipynb` notebooks.
- `output_dir`: A sub-directory called `model_params['modelName']` (`Scenario_A`) will be created in this directory, where all outputs generated by this notebook will be written. 
- `temp_dir`: A temporary directory where model checkpoints are created. This directory should exist and be empty.
- `modis_csv`: The file containing extracted MODIS data for each sample, created by `Extract MODIS Data.ipynb`
- `prism_csv`: The file containing extracted PRISM data for each sample, created by `Extract PRISM Data.ipynb`
- `aux_csv`: The file containing extracted sample labels, DEM, climate zone and other auxiliary data, created by `Extract Auxiliary Data.ipynb`.

In [2]:
input_dir = r'G:\My Drive\LFMC Data\LFMC_ensembles\Datasets'
output_dir = r'G:\My Drive\LFMC Data\LFMC_ensembles\Models'
temp_dir = r'C:\Temp\LFMC'
modis_csv = os.path.join(input_dir, 'modis_365days.csv')
prism_csv = os.path.join(input_dir, 'prism_365days.csv')
aux_csv = os.path.join(input_dir, 'samples_365days.csv')

## Set up experiment parameters
If the experiment dictionary contains a 'tests' key that is not 'falsy' (False, None, 0, empty list) it is assumed to be a list of tests to run. Each test will run with the specified model parameters. Model parameters not specified will be the same for each test, as set in the main model_params dictionary. A failed run can be restarted by setting the 'restart' key to the test that failed. This test and the remaining tests will then be run.

If 'tests' is 'falsy' then a single test will be run using the parameters in the main model_params dictionary.

Other settings are:
- layerTypes: specifies which layers to include in the model
- Layer parameters should be specified as a list. The first entry in the list will be used for the first layer, etc.
- If the experiment includes changes to the layers, all non-default layer parameters need to be included. The parameters that are kept constant can be specified by including a key for the layer type in the experiment dictionary, and the value set to a dictionary of the constant parameters.

Model_parameters that cannot be changed in tests are:
- \*Filename
- \*Channels
- targetColumn

Example of setting layer parameters:
```
{'name': 'Filters',
 'description': 'Test effect of different filter sizes on conv layers',
 'tests': [{'conv': {'filters': [32, 32, 32]}},
           {'conv': {'filters': [8, 8, 8]}},
           {'conv': {'filters': [32, 8, 8]}},
           {'conv': {'filters': [8, 32, 8]}},
           {'conv': {'filters': [8, 8, 32]}},
           {'conv': {'filters': [8, 16, 32]}},
           {'conv': {'filters': [32, 16, 8]}}],
 'conv': {'numLayers': 3, 'poolSize': [2, 3, 4]},
 'restart': 0}
```

In [3]:
experiment = {
    'name': 'yearly_timings',
    'description': 'Timings for yearly architecture changes',
    'layerTypes': ['modisConv', 'prismConv', 'fc'],
    'tests': [
        {},
        {'modisConv': {'numLayers': 5, 'filters': [32] * 5, 'poolSize': [0, 5, 2, 3, 4]},
         'prismConv': {'numLayers': 5, 'filters': [32] * 5, 'poolSize': [0, 5, 2, 3, 4]}
        },
        {'modisConv': {'numLayers': 3, 'filters': [8, 8, 8], 'poolSize': [2, 3, 4]},
         'prismConv': {'numLayers': 3, 'filters': [8, 8, 8], 'poolSize': [2, 3, 4]}
        },
        {'fc': {'numLayers': 2, 'units': [512, 512]}},
        {'fc': {'numLayers': 3, 'units': [256, 256, 256]}},
        {'dropoutRate': 0.5},
        {'batchSize': 32},
    ],
    'restart': 1,
    'testNames': [
        'Proposed model',
        'Conv filters: 32',
        'Conv layers: 3',
        'Dense layers: 2',
        'Dense units: 256',
        'Dropout: 0.5',
        'Batch size: 32',
    ]
}

# Save and display experiment details
experiment_dir = os.path.join(output_dir, experiment['name'])
restart = experiment.get('restart')
if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)
elif not restart:
    raise FileExistsError(f'{experiment_dir} exists but restart not requested')
experiment_file = f'experiment{restart}.json' if restart else 'experiment.json'
with open(os.path.join(experiment_dir, experiment_file), 'w') as f:
    json.dump(experiment, f, indent=2)
experiment

{'name': 'yearly_timings',
 'description': 'Timings for yearly architecture changes',
 'layerTypes': ['modisConv', 'prismConv', 'fc'],
 'tests': [{},
  {'modisConv': {'numLayers': 5,
    'filters': [32, 32, 32, 32, 32],
    'poolSize': [0, 5, 2, 3, 4]},
   'prismConv': {'numLayers': 5,
    'filters': [32, 32, 32, 32, 32],
    'poolSize': [0, 5, 2, 3, 4]}},
  {'modisConv': {'numLayers': 3, 'filters': [8, 8, 8], 'poolSize': [2, 3, 4]},
   'prismConv': {'numLayers': 3, 'filters': [8, 8, 8], 'poolSize': [2, 3, 4]}},
  {'fc': {'numLayers': 2, 'units': [512, 512]}},
  {'fc': {'numLayers': 3, 'units': [256, 256, 256]}},
  {'dropoutRate': 0.5},
  {'batchSize': 32}],
 'restart': 1,
 'testNames': ['Proposed model',
  'Conv filters: 32',
  'Conv layers: 3',
  'Dense layers: 2',
  'Dense units: 256',
  'Dropout: 0.5',
  'Batch size: 32']}

## Set up model parameters
### Architecture for yearly Scenario
These are the required settings for the architecture for the yearly scenario

In [4]:
model_params = ModelParams(modis_layers=5, prism_layers=5, fc_layers=3)

model_params['modelClass'] = 'LfmcTempCnn'
model_params['dataSources'] = ['modis', 'prism', 'aux'] 
model_params['modisFilename'] = modis_csv
model_params['prismFilename'] = prism_csv
model_params['auxFilename'] = aux_csv
model_params['auxColumns'] = ['Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos',
                              'Long_sin', 'Long_cos', 'Lat_norm']
model_params['auxOneHotCols'] = ['Czone3']
model_params['auxAugment'] = True
model_params['splitMethod'] = 'byYear'
model_params['splitYear'] = 2017
model_params['splitFolds'] = 1
model_params['dropoutRate'] = 0.1
model_params['epochs'] = 100
model_params['batchSize'] = 512
model_params['modelRuns'] = 20

model_params['fc'][0]['units'] = 512
model_params['fc'][1]['units'] = 512
model_params['fc'][2]['units'] = 512

model_params['modisConv'][0]['poolSize'] = 0
model_params['modisConv'][1]['poolSize'] = 5
model_params['modisConv'][2]['poolSize'] = 2
model_params['modisConv'][3]['poolSize'] = 3
model_params['modisConv'][4]['poolSize'] = 4

model_params['prismConv'][0]['poolSize'] = 0
model_params['prismConv'][1]['poolSize'] = 5
model_params['prismConv'][2]['poolSize'] = 2
model_params['prismConv'][3]['poolSize'] = 3
model_params['prismConv'][4]['poolSize'] = 4


### Other model parameters
These parameters can be changed. To find out more about any parameter, run `model_params.help('<parameter>')`. 

In [5]:
model_params['modelName'] = experiment['name']
model_params['description'] = experiment['description']
model_params['seedList'] = [
    441, 780, 328, 718, 184, 372, 346, 363, 701, 358,
    566, 451, 795, 237, 788, 185, 397, 530, 758, 633,
    632, 941, 641, 519, 162, 215, 578, 919, 917, 585,
    914, 326, 334, 366, 336, 413, 111, 599, 416, 230,
    191, 700, 697, 332, 910, 331, 771, 539, 575, 457
]

model_params['tempDir'] = temp_dir
model_params['modelDir'] = os.path.join(output_dir, model_params['modelName'])

if not os.path.exists(model_params['modelDir']):
    os.makedirs(model_params['modelDir'])

model_params

{'modelName': 'yearly_timings',
 'description': 'Timings for yearly architecture changes',
 'modelClass': 'LfmcTempCnn',
 'modelDir': 'G:\\My Drive\\LFMC Data\\LFMC_ensembles\\Models\\yearly_timings',
 'tempDir': 'C:\\Temp\\LFMC',
 'diagnostics': False,
 'dataSources': ['modis', 'prism', 'aux'],
 'restartRun': 2,
 'saveModels': False,
 'saveTrain': None,
 'plotModel': True,
 'randomSeed': 1234,
 'modelSeed': 1234,
 'modelRuns': 2,
 'resplit': False,
 'seedList': [441,
  780,
  328,
  718,
  184,
  372,
  346,
  363,
  701,
  358,
  566,
  451,
  795,
  237,
  788,
  185,
  397,
  530,
  758,
  633,
  632,
  941,
  641,
  519,
  162,
  215,
  578,
  919,
  917,
  585,
  914,
  326,
  334,
  366,
  336,
  413,
  111,
  599,
  416,
  230,
  191,
  700,
  697,
  332,
  910,
  331,
  771,
  539,
  575,
  457],
 'maxWorkers': 1,
 'deterministic': False,
 'gpuDevice': 0,
 'gpuMemory': 0,
 'modisFilename': 'G:\\My Drive\\LFMC Data\\LFMC_ensembles\\Datasets\\prism_365days.csv',
 'modisChannels'

## Prepare the data

In [6]:
modis_data = pd.read_csv(modis_csv, index_col=0)
x_modis = reshape_data(np.array(modis_data), model_params['modisChannels'])
print(f'Modis shape: {x_modis.shape}')

prism_data = pd.read_csv(prism_csv, index_col=0)
x_prism = reshape_data(np.array(prism_data), model_params['prismChannels'])
print(f'Prism shape: {x_prism.shape}')

aux_data = pd.read_csv(aux_csv, index_col=0)
y = aux_data[model_params['targetColumn']]

Modis shape: (66946, 365, 7)
Prism shape: (66946, 365, 7)


## Build and run the models
Builds and trains the LFMC models. After training each model, several derived models are created and evaluated. The full list of models is:
- `base` - The fully trained model
- `best` - A model using the checkpoint with the best training loss
- `merge10` - A model created by merging the last 10 checkpoints. The checkpoints are merged by averaging the corresponding weights from each model.
- `ensemble10` - An ensembled model of the last 10 checkpoints. This model averages the predictions made by each model in the ensemble to make the final prediction.
- `merge_best10` - Similar to the merge10 model, but uses the 10 checkpoints with the lowest training/validation losses.

All models, predictions, evaluation statistics, and plots of test results are saved to `model_dir`, with each test and run saved to a separate sub-directory. For each model created, predictions and evaluation statistics are also returned as attributes of the `model` object. These are stored as nested lists, the structure for a full experiment is:
- Tests (omitted if not an experiment)
  - Runs (omitted for a single run)
    - Folds (for k-fold splitting)

In [7]:
def is_experiment():
    try:
        return bool(experiment['tests'])
    except:
        return False

In [8]:
X = {'modis': x_modis, 'prism': x_prism}
if is_experiment():
    ex_models = run_experiment(experiment, model_params, aux_data, X, y)
else:
    print('Running a single test')
    with open(os.path.join(model_params['modelDir'], 'model_params.json'), 'w') as f:
        model_params.save(f)
    models = run_test(model_params, aux_data, X, y)

Experiment yearly_timings - Timings for yearly architecture changes


----------------------------------------------------------------------

test 1 - {'modisConv': {'numLayers': 5, 'filters': [32, 32, 32, 32, 32], 'poolSize': [0, 5, 2, 3, 4]}, 'prismConv': {'numLayers': 5, 'filters': [32, 32, 32, 32, 32], 'poolSize': [0, 5, 2, 3, 4]}}

Auxiliary columns: ['Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos', 'Long_sin', 'Long_cos', 'Lat_norm']
modis shape: (66946, 365, 7)
prism shape: (66946, 365, 7)
aux shape: (66946, 36)

----------------------------------------------------------------------

test 2 - {'modisConv': {'numLayers': 3, 'filters': [8, 8, 8], 'poolSize': [2, 3, 4]}, 'prismConv': {'numLayers': 3, 'filters': [8, 8, 8], 'poolSize': [2, 3, 4]}}

Auxiliary columns: ['Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos', 'Long_sin', 'Long_cos', 'Lat_norm']
modis shape: (66946, 365, 7)
prism shape: (66946, 365, 7)
aux shape: (66946, 36)
yearly_timings_test2_run0 training results: minLoss

## Display the training times
Time are Tensorflow/Keras model training time and excludes data preparation time.

In [22]:
train_times = []
for test in ex_models:
    run_time = 0
    for run in test:
        try:
            run_time += run.train_result['runTime']
        except:
            run_time += pd.read_csv(os.path.join(run.model_dir, 'train_stats.csv')).trainTime[0]
    train_times.append([run_time/60, run_time/60/len(test)])
pd.DataFrame(train_times, index=experiment['testNames'], columns=['ensemble_time', 'single_time'])

,ensemble_time,single_time
Proposed model,508.059500,10.161190
Conv filters: 32,55.672167,27.836083
Conv layers: 3,15.806000,7.903000
Dense layers: 2,19.485500,9.742750
Dense units: 256,19.473500,9.736750
Dropout: 0.5,20.234500,10.117250
Batch size: 32,68.341000,34.170500
